# Build PostGIS DB From NYC OpenData

## Sources

✅ [MapPLUTO](https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_mappluto_20v3_fgdb.zip)
[]()  
✅ [DOITT 3D Buildings](http://maps.nyc.gov/download/3dmodel/DA_WISE_Multipatch.zip) | [docs](https://github.com/CityOfNewYork/nyc-geo-metadata/blob/master/Metadata/Metadata_3DBuildingModel.md)  
✅ [DOITT Planimetrics](https://data.cityofnewyork.us/download/wt4d-p43d/application%2Fzip) | [docs](https://github.com/CityOfNewYork/nyc-planimetrics/blob/master/Capture_Rules.md)  
- hydrography as geom only
- roadbed as geom only
- sidewalk as geom only  

✅ [DPR Street Tree](https://data.cityofnewyork.us/api/geospatial/pi5s-9p35?method=export&format=Original) | [doc](https://data.cityofnewyork.us/api/views/pi5s-9p35/files/2e1e0292-20b4-4678-bea5-6936180074b3?download=true&filename=StreetTreeCensus2015TreesDataDictionary20161102.pdf)

[DCP Zone Boundaries]()  
[DOT CityBench](http://www.nyc.gov/html/dot/downloads/misc/2016-citybench.zip) | [docs](https://data.cityofnewyork.us/Transportation/City-Bench-Locations/8d5p-rji6)
[DOT Bus Shelters](https://data.cityofnewyork.us/api/geospatial/qafz-7myz?method=export&format=Original) | [docs](https://data.cityofnewyork.us/api/views/qafz-7myz/files/2475535f-bda8-4ae1-a74a-3ddbd0ffbc5d?download=true&filename=NYC_Open_Data-Dictionary_-_BusShelters.xlsx)  
[DOT Bicycle Parking Shelters](https://data.cityofnewyork.us/api/geospatial/thbt-gfu9?method=export&format=Original) | only a few features...  
[DOT Bike Parking](http://www.nyc.gov/html/dot/downloads/misc/2013-cityracks-shp.zip) |   
[LinkNYC Locations](https://data.cityofnewyork.us/api/views/s4kf-3yrf/rows.csv?accessType=DOWNLOAD) | CSV  
[DSNY Litter Baskets](https://data.cityofnewyork.us/api/geospatial/uhim-nea2?method=export&format=Shapefile) | no recycling  
[DSNY Recycle Bins](https://data.cityofnewyork.us/api/views/sxx4-xhzg/rows.csv?accessType=DOWNLOAD) |  
[DEP Hydrants](https://data.cityofnewyork.us/api/geospatial/6pui-xhxz?method=export&format=Shapefile) | [docs](https://data.cityofnewyork.us/api/views/6pui-xhxz/files/a0268c0e-38d8-4b48-8bb1-2a2d775b85b5?download=true&filename=DEP-Hydrants__Data_Dictionary_121019.xlsx)

All are uploaded to the database as EPSG=2263 (NY State Plane)

In [25]:
import glob
import fiona
import geopandas as gpd
import pandas as pd
# from cartoframes.auth import set_default_credentials
# from cartoframes import to_carto, has_table, delete_table
import georasters as gr
import numpy as np
from shapely.geometry import MultiPolygon
from shapely import wkt, wkb
import shapely.ops
import os
# from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import create_engine
import geopandas_postgis
from datetime import datetime
from dotenv import load_dotenv, find_dotenv

In [22]:
# load environment variables for postgis connection
load_dotenv(find_dotenv())
conn_vars = ['DB_USER', 'DB_PASSWORD', 'DB_HOST', 'DB_PORT', 'DB_NAME']
user, password, host, port, dbname = [os.getenv(var) for var in conn_vars]

In [23]:
# format connection string
conn_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'

## MapPLUTO

In [54]:
mappluto_path = "/Users/carstenrodin/Desktop/data-src/nyc_mappluto_20v3_fgdb/MapPLUTO20v3.gdb/"
mappluto_gdf = gpd.read_file(mappluto_path)

In [57]:
mappluto_gdf

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,...,FIRM07_FLAG,PFIRM15_FLAG,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Length,Shape_Area,geometry
0,MN,1097,11,104.0,129,1001,02,3.0,10019.0,E054,...,None,None,20v3,None,40.765999,-73.995888,None,1516.586853,112892.966455,"MULTIPOLYGON (((985681.146 218305.545, 985582...."
1,BK,5527,50,311.0,246,2000,20,47.0,11204.0,E247,...,None,None,20v3,None,40.618755,-73.984461,None,240.542515,1886.597524,"MULTIPOLYGON (((988603.467 164740.225, 988540...."
2,BX,3251,421,207.0,409,2000,10,11.0,10468.0,L046,...,None,None,20v3,t,40.876873,-73.888535,None,626.788403,20384.789265,"MULTIPOLYGON (((1015172.497 258742.899, 101503..."
3,QN,15100,175,484.0,1072.02,1006,27,31.0,11693.0,E266,...,1,1,20v3,t,40.635564,-73.823026,None,2091.812869,142215.984020,"MULTIPOLYGON (((1033442.342 170926.651, 103344..."
4,QN,12882,20,413.0,616.01,3000,29,31.0,11422.0,L165,...,None,None,20v3,None,40.679876,-73.730138,None,813.274279,23681.100545,"MULTIPOLYGON (((1059021.836 187078.195, 105902..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857167,BX,4974,2,212.0,456,2000,11,12.0,10475.0,None,...,1,1,20v3,t,40.887855,-73.824285,None,55.161087,49.800158,"MULTIPOLYGON (((1032829.101 262808.126, 103283..."
857168,BK,1619,7,303.0,279,3000,16,36.0,11221.0,E214,...,None,None,20v3,None,40.689498,-73.939068,None,204.506431,1666.555650,"MULTIPOLYGON (((1001190.171 190481.277, 100110..."
857169,QN,4817,22,407.0,1157,1004,25,19.0,11354.0,L144,...,None,None,20v3,None,40.770045,-73.816599,None,285.750910,4124.183427,"MULTIPOLYGON (((1035070.774 219928.879, 103506..."
857170,BX,3039,9,206.0,385,2001,10,15.0,10458.0,E048,...,None,None,20v3,None,40.856649,-73.893269,None,237.090192,1562.016718,"MULTIPOLYGON (((1013806.721 251438.028, 101375..."


In [55]:
# split into chunked list
size = 10000
upload_list = [mappluto_gdf.iloc[i:i+size-1,:] for i in range(0, len(bldgs_gdf),size)]

In [64]:
# for col in upload_list[0].columns:
#     print(col)

In [62]:
upload_list[0].head()[['BBL', 'Address', 'Landmark', 'BldgClass', 'geometry']]

,BBL,Address,Landmark,BldgClass,geometry
0,1.010970e+09,683 11 AVENUE,None,U2,"MULTIPOLYGON (((985681.146 218305.545, 985582...."
1,3.055270e+09,1965 62 STREET,None,A5,"MULTIPOLYGON (((988603.467 164740.225, 988540...."
2,2.032510e+09,WEST 205 STREET,None,U7,"MULTIPOLYGON (((1015172.497 258742.899, 101503..."
3,4.151000e+09,159 STREET,None,U7,"MULTIPOLYGON (((1033442.342 170926.651, 103344..."
4,4.128820e+09,129 AVENUE,None,U2,"MULTIPOLYGON (((1059021.836 187078.195, 105902..."


In [63]:
# upload chunks
for i, gdf in enumerate(upload_list):
    gdf[['BBL', 'Address', 'Landmark', 'BldgClass', 'geometry']].postgis.to_postgis(con=engine, table_name='dcp_mappluto_2020v3', if_exists='append', index=False, geometry='MultiPolygon')
    print('finished: ', i, ' at ', datetime.now().time())

finished:  0  at  18:32:58.735406
finished:  1  at  18:34:14.793581
finished:  2  at  18:35:29.700407
finished:  3  at  18:36:41.872252
finished:  4  at  18:37:54.897083
finished:  5  at  18:39:10.049672
finished:  6  at  18:40:28.643471
finished:  7  at  18:41:47.977552
finished:  8  at  18:43:03.552241
finished:  9  at  18:46:03.510271
finished:  10  at  18:47:16.693731
finished:  11  at  18:48:39.657727
finished:  12  at  18:50:03.646647
finished:  13  at  18:51:37.435258
finished:  14  at  18:52:54.394122
finished:  15  at  18:54:08.002064
finished:  16  at  18:55:21.297236
finished:  17  at  18:56:29.729194
finished:  18  at  18:58:00.470814
finished:  19  at  18:59:07.980492
finished:  20  at  19:00:20.997318
finished:  21  at  19:03:08.817605
finished:  22  at  19:04:23.293452
finished:  23  at  19:05:37.102998
finished:  24  at  19:06:52.043747
finished:  25  at  19:07:58.026562
finished:  26  at  19:09:08.444015
finished:  27  at  19:10:17.564712
finished:  28  at  19:11:23.35

## Buildings

In [3]:
bldg_gdb_paths = glob("/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/*/")

In [4]:
# load buildings from each separate file-geodatabase in the search path into a list
bldg_gdf_list = []
for i in bldg_gdb_paths:
    print(i)
    i_gdf = gpd.read_file(i, driver='FileGDB', layer='Buildings_3D_Multipatch')
    bldg_gdf_list.append(i_gdf.to_crs(epsg=2263))

/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA14_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA11_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA1_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA4_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA10_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA15_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA5_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA20_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Desktop/data-src/DA_WISE_Multipatch/DA_WISE_Multipatch/DA3_3D_Buildings_Multipatch.gdb/
/Users/carstenrodin/Des

In [5]:
# concatenate them all together
bldgs_gdf = pd.concat(bldg_gdf_list) 
bldgs_gdf['BIN'].size # check size

1083529

In [19]:
# make 2d convex hulls of each geometry to use as index
bldgs_gdf['geometry2d'] = bldgs_gdf['geometry'].convex_hull.apply(lambda g: shapely.ops.transform(lambda x, y, z=None: (x, y), g ) )

In [20]:
# check the dataframe
bldgs_gdf

,BIN,DOITT_ID,SOURCE_ID,geometry,geom2d,geometry2d
0,2003044.0,463864.0,1.421000e+10,MULTIPOLYGON Z (((1004374.858 242209.623 80.20...,"POLYGON ((1004361.362 242186.511, 1004293.405 ...","POLYGON ((1004361.362 242186.511, 1004293.405 ..."
1,2002265.0,246383.0,1.421000e+10,MULTIPOLYGON Z (((1008051.732 241826.032 29.18...,"POLYGON ((1008063.019 241779.067, 1008058.991 ...","POLYGON ((1008063.019 241779.067, 1008058.991 ..."
2,2002324.0,745618.0,1.421000e+10,MULTIPOLYGON Z (((1008708.150 243240.652 83.64...,"POLYGON ((1008703.373 243220.920, 1008699.623 ...","POLYGON ((1008703.373 243220.920, 1008699.623 ..."
3,2007216.0,274946.0,1.421000e+10,MULTIPOLYGON Z (((1008788.692 243465.073 73.35...,"POLYGON ((1008812.480 243454.301, 1008803.901 ...","POLYGON ((1008812.480 243454.301, 1008803.901 ..."
4,2002364.0,288278.0,1.421000e+10,MULTIPOLYGON Z (((1008891.149 243139.901 90.63...,"POLYGON ((1008932.034 243099.140, 1008883.234 ...","POLYGON ((1008932.034 243099.140, 1008883.234 ..."
...,...,...,...,...,...,...
66230,4211517.0,372277.0,8.210050e+09,MULTIPOLYGON Z (((1044239.193 197785.779 83.56...,"POLYGON ((1044198.829 197762.606, 1044195.751 ...","POLYGON ((1044198.829 197762.606, 1044195.751 ..."
66231,4161377.0,334592.0,8.210052e+09,MULTIPOLYGON Z (((1053582.062 213351.733 150.2...,"POLYGON ((1053547.761 213341.334, 1053542.173 ...","POLYGON ((1053547.761 213341.334, 1053542.173 ..."
66232,4161105.0,303521.0,8.210054e+09,MULTIPOLYGON Z (((1052411.139 214764.126 98.53...,"POLYGON ((1052423.850 214736.350, 1052411.139 ...","POLYGON ((1052423.850 214736.350, 1052411.139 ..."
66233,4161369.0,805774.0,8.210055e+09,MULTIPOLYGON Z (((1053505.485 213580.658 150.0...,"POLYGON ((1053463.537 213567.906, 1053458.667 ...","POLYGON ((1053463.537 213567.906, 1053458.667 ..."


In [29]:
# create db connection
engine = create_engine(conn_string)

In [22]:
# split into chunked list
size = 10000
upload_list = [bldgs_gdf.iloc[i:i+size-1,:] for i in range(0, len(bldgs_gdf),size)]

In [36]:
# upload chunks
for i, uf in enumerate(upload_list):
    uf[['BIN', 'geometry']].postgis.to_postgis(con=engine, table_name='doitt_3dbldgs_v2016', if_exists='append', index=False, geometry='MultiPolygonZ')
    print('finished: ', i, ' at ', datetime.now().time())

finished:  0  at  08:08:56.011923
finished:  1  at  08:10:04.781242
finished:  2  at  08:11:19.811517
finished:  3  at  08:12:28.472759
finished:  4  at  08:13:39.492386
finished:  5  at  08:14:54.536799
finished:  6  at  08:16:03.324730
finished:  7  at  08:17:09.662465
finished:  8  at  08:18:22.188026
finished:  9  at  08:19:37.304120
finished:  10  at  08:20:48.928520
finished:  11  at  08:22:03.688126
finished:  12  at  08:23:18.627668
finished:  13  at  08:24:29.050021
finished:  14  at  08:25:39.407872
finished:  15  at  08:26:49.319178
finished:  16  at  08:28:03.878954
finished:  17  at  08:29:10.245753
finished:  18  at  08:30:20.415271
finished:  19  at  08:31:34.910884
finished:  20  at  08:32:44.429898
finished:  21  at  08:33:57.183968
finished:  22  at  08:35:16.002539
finished:  23  at  08:36:29.958196
finished:  24  at  08:37:48.897137
finished:  25  at  08:39:04.474027
finished:  26  at  08:40:19.342368
finished:  27  at  08:41:33.010382
finished:  28  at  08:42:43.90

## Import Street Tree Points

In [38]:
street_tree_path = "/Users/carstenrodin/Desktop/data-src/SHP/2015StreetTreesCensus_TREES.shp"
street_tree_gdf = gpd.read_file(street_tree_path)

In [39]:
# fill na values for species
street_tree_gdf['spc_common'].fillna('unknown', inplace=True)

In [40]:
# code tree species as integer
treecode_dict = {tree: i+1 for i, tree in enumerate(sorted(list(street_tree_gdf['spc_common'].unique())))}

In [41]:
# create coded field for species instead of string
street_tree_gdf['spc_code'] = street_tree_gdf.apply(lambda row: treecode_dict[row['spc_common']], axis=1)

In [46]:
# split into chunked list
size = 10000
upload_list = [street_tree_gdf.iloc[i:i+size-1,:] for i in range(0, len(bldgs_gdf),size)]

In [44]:
# to_carto(street_tree_gdf[['geometry', 'spc_code', 'tree_dbh']], 'dpr_streettrees_v2016', if_exists="replace")

In [47]:
for i, f in enumerate(upload_list):
    f[['geometry', 'spc_code', 'tree_dbh']].postgis.to_postgis(con=engine, table_name='dpr_streettrees_v2016', if_exists='append', index=False, geometry='Point')
    print('finished: ', i, ' at ', datetime.now().time())

finished:  0  at  15:24:09.783735
finished:  1  at  15:25:26.038213
finished:  2  at  15:26:32.929632
finished:  3  at  15:27:33.777266
finished:  4  at  15:28:36.635906
finished:  5  at  15:29:48.204880
finished:  6  at  15:30:55.548409
finished:  7  at  15:32:01.116563
finished:  8  at  15:33:32.941881
finished:  9  at  15:34:59.511626
finished:  10  at  15:36:05.086698
finished:  11  at  15:37:08.129671
finished:  12  at  15:38:12.424092
finished:  13  at  15:39:23.266583
finished:  14  at  15:40:31.929902
finished:  15  at  15:41:33.560055
finished:  16  at  15:42:38.269965
finished:  17  at  15:43:46.765798
finished:  18  at  15:45:05.561922
finished:  19  at  15:46:22.610143
finished:  20  at  15:47:35.141513
finished:  21  at  15:48:33.685473
finished:  22  at  15:49:32.831494
finished:  23  at  15:50:35.964639
finished:  24  at  15:51:40.813893
finished:  25  at  15:52:42.725300
finished:  26  at  15:53:44.227850
finished:  27  at  15:54:47.669838
finished:  28  at  15:55:52.62

## Import Elevation

This is currently done by sampling a DEM raster from a regularly spaced grid of points in QGIS, then uploading those points

In [ ]:
street_intersection_path = "/Users/carstenrodin/local-git/ud-3d-model/in/mn_regularpts_100/mn_regularpts_100_raster.shp"
street_intersection_gdf = gpd.read_file(street_intersection_path).to_crs(epsg=4326)

In [ ]:
street_intersection_gdf

In [ ]:
to_carto(street_intersection_gdf, 'ud_elevationpts_v2020', if_exists="replace")

## Planimetrics

In [49]:
# load planimetrics as EPSG:4326
planimetric_gdf_list = {}
planimetric_path = '/Users/carstenrodin/Desktop/data-src/NYC_DoITT_Planimetric_OpenData.gdb/'
planimetric_layers = fiona.listlayers(planimetric_path)
for l in planimetric_layers:
    l_gdf = gpd.read_file(planimetric_path, driver='FileGDB', layer=l)
    planimetric_gdf_list[l] = l_gdf.to_crs(epsg=4326)

In [50]:
planimetric_gdf_list.keys()

dict_keys(['PAVEMENT_EDGE', 'HYDRO_STRUCTURE', 'RETAININGWALL', 'HYDROGRAPHY', 'SIDEWALK', 'PARK', 'MEDIAN', 'SWIMMING_POOL', 'OPEN_SPACE_NO_PARK', 'PARKING_LOT', 'SHORELINE', 'BOARDWALK', 'RAILROAD', 'TRANSPORT_STRUCTURE', 'ELEVATION', 'MISC_STRUCTURE_POLY', 'CURB', 'ROADBED', 'PLAZA', 'SIDEWALK_LINE', 'RAILROAD_STRUCTURE'])

In [51]:
planimetric_gdf_list['SIDEWALK'][['geometry']].postgis.to_postgis(con=engine, table_name='doitt_sidewalk_v2016', if_exists='append', index=False, geometry='MultiPolygon')

In [52]:
planimetric_gdf_list['ROADBED'][['geometry']].postgis.to_postgis(con=engine, table_name='doitt_roadbed_v2016', if_exists='append', index=False, geometry='MultiPolygon')

In [53]:
planimetric_gdf_list['HYDROGRAPHY'][['geometry']].postgis.to_postgis(con=engine, table_name='doitt_hydrography_v2016', if_exists='append', index=False, geometry='MultiPolygon')

## Other Export Options

In [ ]:
# save to file
# bldgs_gdf.to_file("/Users/carstenrodin/Desktop/bldgs.gpkg", driver="GPKG")